In [245]:
from scipy.special import iv
import numpy as np
import timeit
import viscid
import imp

In [246]:
#If you edit viscid while the kernel is running, run this to reload the package
imp.reload(viscid)

<module 'viscid' from '/Users/zack/Desktop/shaker_docs/faraday/viscid.py'>

In [247]:
args=viscid.args

In [248]:
kx=0
ky=args.k2y/2
omega=2*np.pi*args.freq/2

start=timeit.default_timer()
E = np.zeros((3,3,2*args.Nt+1,2*args.Nt+1,2*args.Nx+1,2*args.Nx+1,2*args.Ny+1,2*args.Ny+1),dtype=np.complex128)
#mode indices. axes appear in the order (l',l,m',m,n',n)
lps=np.arange(-args.Nt,args.Nt+1)[:,np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
ls=np.arange(-args.Nt,args.Nt+1)[np.newaxis,:,np.newaxis,np.newaxis,np.newaxis,np.newaxis]
mps=np.arange(-args.Nx,args.Nx+1)[np.newaxis,np.newaxis,:,np.newaxis,np.newaxis,np.newaxis]
ms=np.arange(-args.Nx,args.Nx+1)[np.newaxis,np.newaxis,np.newaxis,:,np.newaxis,np.newaxis]
nps=np.arange(-args.Ny,args.Ny+1)[np.newaxis,np.newaxis,np.newaxis,np.newaxis,:,np.newaxis]
ns=np.arange(-args.Ny,args.Ny+1)[np.newaxis,np.newaxis,np.newaxis,np.newaxis,np.newaxis,:]

kappax = kx + args.k1x*ms + args.k2x*ns
kappay = ky + args.k1y*ms + args.k2y*ns
kappa = (kappax**2+kappay**2)**0.5
Omega = 1j*(omega + 2*np.pi*args.freq*ls) + args.mu/args.rho*kappa**2

#Define also here the symmetric sums and differences C,Ctile,S,Stilde above Eq. 31, and then assign the corresponding elements of E with array slicing.
Ctilde = np.exp(-(args.rho/args.mu*Omega)**0.5*args.h0)*iv(ms-mps,(args.rho/args.mu*Omega)**0.5*args.As/2)*iv(ns-nps,(args.rho/args.mu*Omega)**0.5*args.As/2) + np.exp((args.rho/args.mu*Omega)**0.5*args.h0)*iv(ms-mps,-(args.rho/args.mu*Omega)**0.5*args.As/2)*iv(ns-nps,-(args.rho/args.mu*Omega)**0.5*args.As/2)
C = np.exp(-kappa*args.h0)*iv(ms-mps,kappa*args.As/2)*iv(ns-nps,kappa*args.As/2) + np.exp(kappa*args.h0)*iv(ms-mps,-kappa*args.As/2)*iv(ns-nps,-kappa*args.As/2)

E[0,0][(np.arange(E.shape[2]),np.arange(E.shape[3]))] = Ctilde-2*args.mu/args.rho*kappax**2*C/(args.mu*kappa**2+args.rho*Omega)

stop=timeit.default_timer()
print(stop-start)

0.007680631999392062


In [251]:
E=viscid.viscid_mat(2*np.pi*args.freq/2,0,args.k2y/2)
E.shape
flattened=np.transpose(E,(0,2,4,6,1,3,5,7)).reshape((3*(2*args.Nx+1)*(2*args.Ny+1)*(2*args.Nt+1),3*(2*args.Nx+1)*(2*args.Ny+1)*(2*args.Nt+1)))
flattened.shape

(1029, 1029)

In [255]:
E[0,0,3,3,3,3,3,3]

(-34.43596924438191-39.92555860385018j)

In [257]:
flattened[0]

array([-6717.07354657-2745.15609605j,  2125.55748825 -773.77625834j,
        -157.13972678 +318.63891143j, ...,     0.           +0.j        ,
           0.           +0.j        ,     0.           +0.j        ])

In [41]:
#some test for kronecker deltas
start=timeit.default_timer()
deltal=np.ones((ls-lps).shape)
deltal[np.nonzero(ls-lps)]=0
deltam=np.ones((ms-mps).shape)
deltam[np.nonzero(ms-mps)]=0
deltan=np.ones((ns-nps).shape)
deltan[np.nonzero(ns-nps)]=0
E[0,0]=(deltan*deltam*deltal)
stop=timeit.default_timer()
print(stop-start)

#flattening into a 2d array requires reshaping so the right indices correspond
start=timeit.default_timer()
flattened=np.transpose(E,(0,2,4,6,1,3,5,7)).reshape((3*(2*args.Nx+1)*(2*args.Ny+1)*(2*args.Nt+1),3*(2*args.Nx+1)*(2*args.Ny+1)*(2*args.Nt+1)))
stop=timeit.default_timer()
print(stop-start)
print(flattened.nonzero())

#If we used the order with all primes followed by all nonprimes, reshaping would be faster
start=timeit.default_timer()
flattened=E.reshape((3*(2*args.Nx+1)*(2*args.Ny+1)*(2*args.Nt+1),3*(2*args.Nx+1)*(2*args.Ny+1)*(2*args.Nt+1)))
stop=timeit.default_timer()
print(stop-start)
print(flattened.nonzero())


0.0075672230000236596
0.17459624299999632
(array([   0,    1,    2, ..., 1328, 1329, 1330]), array([   0,    1,    2, ..., 1328, 1329, 1330]))
0.014356282999870018
(array([  0,   0,   0, ..., 443, 443, 443]), array([   0,   12,   24, ..., 2637, 2649, 2661]))
